In [0]:
%sql
USE CATALOG lab; 
USE SCHEMA data;

# Step 1: Create campaigns_silver table
This table standardizes data types and cleanses the raw campaigns data for downstream analytics and joins.

In [0]:
%sql
CREATE OR REPLACE TABLE campaigns_silver
AS
SELECT
  CAST(campaign_id AS INT) AS campaign_id,
  campaign_name,
  channel,
  TO_DATE(start_date) AS start_date,
  TO_DATE(end_date) AS end_date,
  CAST(budget_usd AS DOUBLE) AS budget_usd,
  objective
FROM lab.data.campaigns;

# Step 2: Create daily_spend_silver table
This table standardizes and cleanses the raw daily spend data, filtering out negative spend values for accurate reporting.

In [0]:
%sql
CREATE OR REPLACE TABLE daily_spend_silver
AS
SELECT
  CAST(campaign_id AS INT) AS campaign_id,
  TO_DATE(date) AS date,
  CAST(spend_usd AS DOUBLE) AS spend_usd
FROM lab.data.daily_spend
WHERE spend_usd >= 0;

# Step 3: Create impressions_silver table
Purpose: This table standardizes and cleanses the raw impressions data for downstream analytics and attribution.

In [0]:
%sql
CREATE OR REPLACE TABLE impressions_silver
AS
SELECT
  CAST(user_id AS INT) AS user_id,          -- cast to INT for consistency
  CAST(campaign_id AS INT) AS campaign_id,  -- cast to INT for consistency
  device,
  country,
  placement,
  TIMESTAMP(event_ts) AS event_ts,          -- convert string timestamp to TIMESTAMP
  TO_DATE(event_ts) AS event_date           -- derive date for partitioning
FROM lab.data.impressions;

# Step 4: Create clicks_silver table
This table standardizes and cleanses the raw clicks data for downstream analytics and attribution.

In [0]:
%sql
CREATE OR REPLACE TABLE clicks_silver
AS
SELECT
  CAST(user_id AS INT) AS user_id,
  CAST(campaign_id AS INT) AS campaign_id,
  device,
  country,
  placement,
  TIMESTAMP(event_ts) AS event_ts,
  TO_DATE(event_ts) AS event_date
FROM lab.data.clicks;

# Step 5: Create customers_silver table
This table standardizes and cleanses the raw customers data for downstream analytics and attribution.

In [0]:
%sql
CREATE OR REPLACE TABLE customers_silver
AS
SELECT
  CAST(user_id AS INT) AS user_id,
  TO_DATE(signup_date) AS signup_date,
  first_touch_channel
FROM lab.data.customers;

# Step 6: Create orders_silver table
This table standardizes and cleanses the raw orders data, filtering out negative revenue values for accurate reporting.

In [0]:
%sql
CREATE OR REPLACE TABLE orders_silver
AS
SELECT
  CAST(user_id AS INT) AS user_id,
  TO_DATE(order_date) AS order_date,
  CAST(revenue_usd AS DOUBLE) AS revenue_usd
FROM lab.data.orders
WHERE revenue_usd >= 0;

# Step 7: Create daily_channel_metrics_gold table
This table aggregates silver layer data to produce daily campaign/channel-level KPIs (spend, impressions, clicks, CTR, CVR, ROAS, etc.) for reporting and analysis.

In [0]:
%sql
-- Gold: Aggregates and KPIs
CREATE OR REPLACE TABLE daily_channel_metrics_gold
AS
WITH impr AS (
  -- Aggregate impressions by date and campaign
  SELECT event_date AS date, campaign_id, COUNT(*) AS impressions
  FROM lab.data.impressions_silver
  GROUP BY 1,2
), clk AS (
  -- Aggregate clicks by date and campaign
  SELECT event_date AS date, campaign_id, COUNT(*) AS clicks
  FROM lab.data.clicks_silver
  GROUP BY 1,2
), sp AS (
  -- Aggregate spend by date and campaign
  SELECT date, campaign_id, SUM(spend_usd) AS spend_usd
  FROM lab.data.daily_spend_silver
  GROUP BY 1,2
), ord AS (
  -- Aggregate revenue by order date and user
  SELECT order_date AS date, user_id, SUM(revenue_usd) AS revenue_usd
  FROM lab.data.orders_silver
  GROUP BY 1,2
), ch AS (
  -- Get campaign-channel mapping
  SELECT campaign_id, channel FROM lab.data.campaigns_silver
)
SELECT
  COALESCE(sp.date, impr.date, clk.date) AS date, -- Use available date from spend, impressions, or clicks
  COALESCE(sp.campaign_id, impr.campaign_id, clk.campaign_id) AS campaign_id, -- Use available campaign_id
  ch.channel, -- Channel name from campaigns_silver
  COALESCE(sp.spend_usd, 0) AS spend_usd, -- Default spend to 0 if missing
  COALESCE(impressions, 0) AS impressions, -- Default impressions to 0 if missing
  COALESCE(clicks, 0) AS clicks, -- Default clicks to 0 if missing
  CASE WHEN impressions > 0 THEN clicks / impressions ELSE 0 END AS ctr, -- Click-through rate
  ord_kpis.revenue_usd, -- Revenue attributed to campaign/date
  CASE WHEN clicks > 0 THEN ord_kpis.orders * 1.0 / clicks ELSE 0 END AS cvr, -- Conversion rate
  CASE WHEN COALESCE(spend_usd, 0) > 0 THEN COALESCE(ord_kpis.revenue_usd, 0) / spend_usd ELSE 0 END AS roas -- Return on ad spend
FROM sp
FULL OUTER JOIN impr ON sp.date = impr.date AND sp.campaign_id = impr.campaign_id -- Join spend and impressions
FULL OUTER JOIN clk ON COALESCE(sp.date, impr.date) = clk.date AND COALESCE(sp.campaign_id, impr.campaign_id) = clk.campaign_id -- Join clicks
LEFT JOIN ch ON COALESCE(sp.campaign_id, impr.campaign_id, clk.campaign_id) = ch.campaign_id -- Join channel info
LEFT JOIN (
  -- Attribution: count orders and sum revenue for users who clicked on a campaign on a given date
  SELECT d.date, d.campaign_id,
         COUNT(DISTINCT o.user_id) AS orders, -- Number of unique users who ordered
         SUM(o.revenue_usd) AS revenue_usd -- Total revenue from those users
  FROM (
    SELECT DISTINCT event_date AS date, campaign_id, user_id
    FROM lab.data.clicks_silver
  ) d
  LEFT JOIN lab.data.orders_silver o ON o.user_id = d.user_id AND o.order_date = d.date -- Attribute orders to click date
  GROUP BY 1,2
) ord_kpis ON ord_kpis.date = COALESCE(sp.date, impr.date, clk.date)
          AND ord_kpis.campaign_id = COALESCE(sp.campaign_id, impr.campaign_id, clk.campaign_id);

# Step 8: Create channel_daily_gold table
This table further aggregates daily_channel_metrics_gold to the channel level for high-level reporting and dashboarding.

In [0]:
%sql
CREATE OR REPLACE TABLE channel_daily_gold
AS
SELECT date, channel,
       SUM(spend_usd) AS spend_usd,
       SUM(impressions) AS impressions,
       SUM(clicks) AS clicks,
       CASE WHEN SUM(impressions) > 0 THEN SUM(clicks) * 1.0 / SUM(impressions) ELSE 0 END AS ctr,
       SUM(revenue_usd) AS revenue_usd,
       CASE WHEN SUM(clicks) > 0 THEN COUNT(*) * 1.0 / SUM(clicks) ELSE 0 END AS cvr_estimate,
       CASE WHEN SUM(spend_usd) > 0 THEN SUM(revenue_usd) / SUM(spend_usd) ELSE 0 END AS roas
FROM lab.data.daily_channel_metrics_gold
GROUP BY 1,2;